In [1]:
import glob
from bs4 import BeautifulSoup
import requests
import string
import pandas as pd
import requests
import os

In [2]:
path = "lexile/raw/lists/"
files = [f for f in glob.glob(path + "*.html", recursive=True)]
base_url = 'https://www.commonsensemedia.org/book-reviews'

In [3]:
def get_titles(soup):
    for s in soup.findAll(class_="views-field views-field-field-reference-review-ent-prod result-title"):
        titles.append(s.get_text().strip())
    return titles

In [4]:
def edit_titles(titles):
    stripper = str.maketrans("", "", string.punctuation)
    titles_edited = [t.translate(stripper).replace(" ", "-").lower() for t in titles]
    return titles_edited

In [5]:
def make_urls(title_edited):
    urls = [base_url + "/" + t for t in title_edited]
    return urls

In [6]:
titles = []

for file in files:
    with open(file, 'r') as f: 
        soup = BeautifulSoup(f.read(), 'html.parser')
        get_titles(soup)
        
titles_edited = edit_titles(titles)

urls = make_urls(titles_edited)

In [7]:
df = pd.DataFrame()
df['title'] = titles
df['title_edited'] = titles_edited
df['url'] = urls
df.loc[:, "title_edited"] = df.loc[:, 'title_edited'].str.replace("--", "-")
df.loc[:, "url"] = df.loc[:, "url"].str.replace("--", "-")
df.head(5)

,title,title_edited,url
0,Joshua Dread,joshua-dread,https://www.commonsensemedia.org/book-reviews/...
1,Abe Lincoln's Dream,abe-lincolns-dream,https://www.commonsensemedia.org/book-reviews/...
2,Teeth,teeth,https://www.commonsensemedia.org/book-reviews/...
3,Nancy Clancy: Secret Admirer,nancy-clancy-secret-admirer,https://www.commonsensemedia.org/book-reviews/...
4,On the Road to Mr. Mineo's,on-the-road-to-mr-mineos,https://www.commonsensemedia.org/book-reviews/...


In [9]:
df.to_csv("~/final_project/lexile/raw/title_urls.csv", index=False)

# Scrape Each Book's Review Page

In [ ]:
!mkdir lexile/raw/books

In [ ]:
for i, j in df.iterrows():
    if not os.path.isfile(path + j.title_edited + ".html"):
        page = requests.get(j.url)
        soup = BeautifulSoup(page.text, 'html.parser')
        with open("lexile/raw/books/" + j.title_edited + ".html", 'wb') as f:
            try:
                f.write(soup.encode('utf-8'))
            except:
                with open("lexile/raw/book-errors.txt", 'a+') as f:
                    f.write(j.title + "\n")
    else: 
        continue

## Identify the gap

In [10]:
df.shape[0]

5816

In [12]:
files = os.listdir("lexile/raw/books")
len(files)

5803

In [30]:
a = [f[:-5] for f in files]
a[0]

'the-third-twin'

In [33]:
b = df.loc[:, "title_edited"]
b[0]

'joshua-dread'

In [39]:
#temp3 = [item for item in temp1 if item not in temp2]
c = [i for i in b  if i not in a]

In [41]:
import numpy as np 

In [42]:
np.setdiff1d(a, b)

array(['', '.ipynb_checkp',
       'a-girl-undone--a-girl-called-fearless-book-2',
       'a-map-of-days--miss-peregrines-peculiar-children-book-4',
       'absolutely-truly--a-pumpkin-falls-mystery',
       'aerie--magonia-book-2', 'agua-aguita--water-little-water',
       'always-and-forever-lara-jean--to-all-the-boys-ive-loved-before-book-3',
       'ashes--the-seeds-of-america-trilogy-book-3',
       'astrotwins--project-rescue-astrotwins-book-2',
       'avenged--ruined-book-2',
       'babymouse-tales-from-the-locker--lights-camera-middle-school',
       'bluecrowne--a-greenglass-house-story',
       'bomb-the-race-to-build--and-steal--the-worlds-most-dangerous-weapon',
       'brave-face--a-memoir', 'broken-throne--a-red-queen-collection',
       'burning-kingdoms--the-internment-chronicles-book-2', 'capt',
       'caravaggio--painter-on-the-run',
       'catstronauts-mission-moon--catstronauts-book-1', 'cerca--close',
       'cookies--bitesize-life-lessons',
       'darkness-br